# Explore whether the Zillow Data has the accurate substation attached

The substation attachment to the Zillow house data was completed before the  start of this project. This repository explores the accuracy of the pre-assingments. 